# ROB2004 Final Project - pushing an object

The goal of this project is to resolve a simple manipulation task.

## Instructions
* Answer all questions in the notebook
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a 2-3 pages report in pdf format (pdf only) detailing the methodology you followed to answer the questions as well as answers to the questions that require a typeset answer. You may add the plots in the report (does not count for the page limit) or in the Jupyter notebook.
* This is an individual project

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Question

Implement a controller using the code skeletong below in order to get the robot to go and pick up the red blocks and drop them in the green bowl. The bowl position is $(-0.3,0.55,0.65)$ (in spatial frame coordinates) and the blocks positions are $(0.35,0.58,0.65)$ and $(0.15,0.67,0.65)$ respectively.

<div>
<img src="./manipulation_scene.jpg" width="300"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy and scipy
* You need to use at least one controller in the end-effector space
* You need to generate smooth motions
* You need to compensate for the gravity of the robot

In your report, describe the controller and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit).

In [ ]:
import numpy as np
import time

import matplotlib.pyplot as plt

from panda import Simulator, PandaRobot

np.set_printoptions(precision=4, suppress = True)

# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. These functions are implemented in the PandaRobot class (in the panda.py file).
Examples are shown below:

In [ ]:
# create an instance of the helper class
my_robot = PandaRobot()

# compute the forward kinematics for an arbitrary joint configuration
q = np.random.uniform(-1,1,7)
pose = my_robot.FK(np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0]))
print(f'the pose of the end-effector for joint configuration\n\n {q}\n\n is\n\n {pose}')

In [ ]:
# we can also get the Jacobian in various frames 
# it is sufficient to pass the name of the frame as an argument as shown below

q = np.random.uniform(-1,1,7)

spatial_jacobian = my_robot.get_jacobian(q, 'S')
body_jacobian = my_robot.get_jacobian(q, 'B')

# we can even get the Jacobian in a frame place at the same position as the end-effector 
# frame but oriented like the spatial frame
oriented_jacobian = my_robot.get_jacobian(q, 'O')

print(f'for joint configuration \n\n {q} \n \n the spatial Jacobian is \n\n {spatial_jacobian}\n\n')
print(f'the body Jacobian is \n\n {body_jacobian}\n\n')
print(f'and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is\n\n {oriented_jacobian}')

In [ ]:
# finally it is possible to compute the effect of gravity on the joints
# this can for example be used to do gravity compensation when controlling the robot

q = np.random.uniform(-1,1,7)

g = my_robot.g(q)

print(f'for joint configuration \n\n {q} \n \n the gravity force seen on the joints is \n \n {g}')

# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs. The class `Simulator` contains all the code for the simulation. The code skeleton currently implement a simple PD controller in joint space that just keeps a desired current position.

Importantly you can control both the joints and the gripper of the robot:
* `simulator.send_joint_torque(joint_torques)` sends a vector of dimension 7 setting the torques of the joints
* `simulator.gripper_move(gripper_position)` sends a vector of dimension 2 setting the position of the fingers of the gripper. The fingers can have positions from 0 to 0.04cm (you can fully close the gripper by setting `simulator.gripper_move([0,0])` and fully open it by setting `simulator.gripper_move([0.04,0.04)`)

In [ ]:
# here we create a simulation and reset the state of the robot
simulator = Simulator()
simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])

# we create a robot object so we can use its helper functions
my_robot = PandaRobot()

# duration of the simulation
run_time = 30.

# simulation time step
dt = 0.005
num_steps = int(run_time/dt)


# we store information
ndofs = 7 # number of degrees of freedom (excluding the gripper)
measured_positions = np.zeros([num_steps,ndofs])
measured_velocities = np.zeros_like(measured_positions)
desired_torques = np.zeros_like(measured_positions)
desired_positions = np.zeros_like(measured_positions)
desired_velocities = np.zeros_like(measured_positions)
time = np.zeros([num_steps])


### this is the main control loop
for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = simulator.get_state()
    measured_positions[i,:] = q
    measured_velocities[i,:] = dq
    
    ## controller: TODO HERE IMPLEMENT YOUR CONTROLLER TO SOLVE THE TASK
    ## you will need to replace the naive PD controller implemented below

    # desired joint positions and velocities
    q_des = np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2,0.0])
    dq_des = np.zeros([7])
    
    # we save the desired positions/velocities for later plotting 
    desired_positions[i,:] = q_des
    desired_velocities[i,:] = dq_des
    
    ##PD controller
    # the PD gains
    P = np.array([100., 100., 100., 100., 100., 100., 50.])
    D = np.array([10.,10,10,10,10,10,1.])

    error = q_des - q # the position error for all the joints
    d_error = dq_des-dq # the velocity error for all the joints
    
    
    # we compute the desired torques as a PD controller
    joint_torques = np.diag(P) @ error + np.diag(D) @ d_error + my_robot.g(q)
    
    # we start by closing the gripper and then open it after 5 seconds
    gripper_position = [0,0]
    if time[i] > 5.0:
        gripper_position = [0.04, 0.04]
    
    # we send them to the robot and do one simulation step
    simulator.send_joint_torque(joint_torques)
    simulator.gripper_move(gripper_position)
    simulator.step()